In [1]:
! git clone https://github.com/OctoberChang/klcpd_code.git

Cloning into 'klcpd_code'...
remote: Enumerating objects: 235, done.
remote: Total 235 (delta 0), reused 0 (delta 0), pack-reused 235
Receiving objects: 100% (235/235), 43.68 MiB | 16.16 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 13.5 MB/s 
     |████████████████████████████████| 125 kB 76.6 MB/s 
     |████████████████████████████████| 512 kB 66.3 MB/s 


In [ ]:
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset

import pytorch_lightning as pl
from typing import List, Tuple
from sklearn.metrics.pairwise import euclidean_distances

import random

#### Utils

In [ ]:
def load_data(data_path):
    data = sio.loadmat(data_path)
    return data['Y'], data['L']

def load_matlab_v1_log(data_path):
    eval_log = sio.loadmat(data_path)
    ret_dict = {'Y_tst': eval_log['Y_tst'],
                'L_tst': eval_log['L_tst'],
                'Y_tst_pred': eval_log['Y_tst_pred'],
                'L_tst_pred': eval_log['err'][:, 1], # use MSE as predict score
                'err': eval_log['err']}
    return ret_dict

def load_matlab_v2_log(data_path):
    eval_log = sio.loadmat(data_path)
    ret_dict = {'Y_tst': eval_log['Y_tst'],
                'L_tst': eval_log['L_tst'],
                'Y_tst_pred': None,
                'L_tst_pred': eval_log['Y_tst_pred'],
                'err': None}
    return ret_dict

def load_python_log(data_path):
    eval_log = pickle.load(open(data_path, 'rb'))
    ret_dict = {'Y_tst': eval_log['Y_true'],
                'L_tst': eval_log['L_true'],
                'Y_tst_pred': eval_log['Y_pred'],
                'L_tst_pred': eval_log['L_pred'],
                'err': None}
    return ret_dict

def compute_auc(eval_dict):
    L_true = eval_dict['L_tst'].flatten()
    L_pred = eval_dict['L_tst_pred'].flatten()
    # print('L_true', L_true.shape, 'L_pred', L_pred.shape)
    fp_list, tp_list, thresholds = sklearn.metrics.roc_curve(L_true, L_pred)
    auc = sklearn.metrics.auc(fp_list, tp_list)
    return fp_list, tp_list, auc

def compute_average_roc(tprs, base_fpr):
    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std
    return mean_tprs

def forecast_loss(eval_dict):
    assert(eval_dict['Y_tst_pred'] is not None)
    sqr_err = np.sum((eval_dict['Y_tst'] - eval_dict['Y_tst_pred'])**2, axis=1)
    abs_err = np.sum(abs(eval_dict['Y_tst'] - eval_dict['Y_tst_pred']), axis=1)
    mse_mean = np.mean(sqr_err)
    mae_mean = np.mean(abs_err)
    return mse_mean, mae_mean

def print_auc_table(result_array, all_methods):
    # print auc for latex table
    print('metric', end='')
    for i, method in enumerate(all_methods):
        print(' & %s' % (method), end='')
    print('')
    print('AUC', end='')
    for i, method in enumerate(all_methods):
        print(' & %.4f' % (np.mean(result_array[i, :])), end='')
    print('')

#### Loss

In [ ]:


# --------------------------------------------------------------------------------------#
#                                          Loss                                         #
# --------------------------------------------------------------------------------------#


#### X = Data.Y_subspace

def median_heuristic(med_sqdist, beta=0.5):
    beta_list = [beta ** 2, beta ** 1, 1, (1.0 / beta) ** 1, (1.0 / beta) ** 2]
    return [med_sqdist * b for b in beta_list]

In [ ]:
def batch_mmd2_loss(X_p_enc, X_f_enc, sigma_var):
    device = X_p_enc.device
    # some constants, TODO ask Alex
    n_basis = 1024
    gumbel_lmd = 1e+6
    cnst = math.sqrt(1. / n_basis)
    n_mixtures = sigma_var.size(0)
    n_samples = n_basis * n_mixtures
    batch_size, seq_len, nz = X_p_enc.size()

    # gumbel trick to get masking matrix to uniformly sample sigma
    def sample_gmm(W, batch_size):
        U = torch.FloatTensor(batch_size * n_samples, n_mixtures).uniform_()
        U = U.to(W.device)
        sigma_samples = F.softmax(U * gumbel_lmd, dim=1).matmul(sigma_var)
        W_gmm = W.mul(1. / sigma_samples.unsqueeze(1))
        W_gmm = W_gmm.view(batch_size, n_samples, nz)
        return W_gmm

    W = torch.FloatTensor(batch_size * n_samples, nz).normal_(0, 1)
    W = W.to(device)
    W.requires_grad = False
    W_gmm = sample_gmm(W, batch_size)  # batch_size x n_samples x nz
    W_gmm = torch.transpose(W_gmm, 1, 2).contiguous()  # batch_size x nz x n_samples

    XW_p = torch.bmm(X_p_enc, W_gmm)  # batch_size x seq_len x n_samples
    XW_f = torch.bmm(X_f_enc, W_gmm)  # batch_size x seq_len x n_samples
    z_XW_p = cnst * torch.cat((torch.cos(XW_p), torch.sin(XW_p)), 2)
    z_XW_f = cnst * torch.cat((torch.cos(XW_f), torch.sin(XW_f)), 2)
    batch_mmd2_rff = torch.sum((z_XW_p.mean(1) - z_XW_f.mean(1)) ** 2, 1)
    return batch_mmd2_rff

In [ ]:
def mmdLossD(X_f,
             Y_f,
             X_f_enc,  # real (initial)   subseq (future window)
             Y_f_enc,  # fake (generated) subseq (future window)
             X_p_enc,  # real (initial)   subseq (past window)
             X_f_dec,
             Y_f_dec,
             lambda_ae,
             lambda_real,
             sigma_var):
    # batchwise MMD2 loss between X_f and Y_f
    D_mmd2 = batch_mmd2_loss(X_f_enc, Y_f_enc, sigma_var)

    # batchwise MMD2 loss between X_p and X_f
    mmd2_real = batch_mmd2_loss(X_p_enc, X_f_enc, sigma_var)

    # reconstruction loss
    real_L2_loss = torch.mean((X_f - X_f_dec) ** 2)
    fake_L2_loss = torch.mean((Y_f - Y_f_dec) ** 2)

    lossD = D_mmd2.mean() - lambda_ae * (real_L2_loss + fake_L2_loss) - lambda_real * mmd2_real.mean()

    return lossD.mean(), mmd2_real.mean()


#### Models

In [ ]:
class NetG(nn.Module):
    def __init__(self, args, data):
        super(NetG, self).__init__()
        self.wnd_dim = args.wnd_dim
        self.var_dim = data.var_dim
        self.D = data.D
        self.RNN_hid_dim = args.RNN_hid_dim

        self.rnn_enc_layer = nn.GRU(self.var_dim, self.RNN_hid_dim, num_layers=1, batch_first=True)
        self.rnn_dec_layer = nn.GRU(self.var_dim, self.RNN_hid_dim, num_layers=1, batch_first=True)
        self.fc_layer = nn.Linear(self.RNN_hid_dim, self.var_dim)

    # X_p:   batch_size x wnd_dim x var_dim (Encoder input)
    # X_f:   batch_size x wnd_dim x var_dim (Decoder input)
    # h_t:   1 x batch_size x RNN_hid_dim
    # noise: 1 x batch_size x RNN_hid_dim
    def forward(self, X_p, X_f, noise):
        X_p_enc, h_t = self.rnn_enc_layer(X_p)
        X_f_shft = self.shft_right_one(X_f)
        hidden = h_t + noise
        Y_f, _ = self.rnn_dec_layer(X_f_shft, hidden)
        output = self.fc_layer(Y_f)
        return output

    def shft_right_one(self, X):
        X_shft = X.clone()
        X_shft[:, 0, :].data.fill_(0)
        X_shft[:, 1:, :] = X[:, :-1, :]
        return X_shft


class NetD(nn.Module):
    def __init__(self, args, data):
        super(NetD, self).__init__()

        self.wnd_dim = args.wnd_dim
        self.var_dim = data.var_dim
        self.D = data.D
        self.RNN_hid_dim = args.RNN_hid_dim

        self.rnn_enc_layer = nn.GRU(self.var_dim, self.RNN_hid_dim, batch_first=True)
        self.rnn_dec_layer = nn.GRU(self.RNN_hid_dim, self.var_dim, batch_first=True)

    def forward(self, X):
        X_enc, _ = self.rnn_enc_layer(X)
        X_dec, _ = self.rnn_dec_layer(X_enc)
        return X_enc, X_dec

In [ ]:

# --------------------------------------------------------------------------------------#
#                                        Models                                        #
# --------------------------------------------------------------------------------------#

# separation for training
def _history_future_separation(data, window):
    history = data[:, :, :window]
    future = data[:, :, window:2 * window]
    return history, future

class KLCPD(pl.LightningModule):
    def __init__(
            self,
            netG: nn.Module,
            netD: nn.Module,
            args: dict,
            train_dataset: Dataset,
            test_dataset: Dataset,
            num_workers: int = 2
    ) -> None:

        super().__init__()
        self.args = args
        self.netG = netG
        self.netD = netD

        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        
        ####
        '''
        def median_heuristic(X, beta=0.5):
            max_n = min(30000, X.shape[0])
            D2 = euclidean_distances(X[:max_n], squared=True)
    

        '''
        #### Added for conformity
        #X = Data.Y_subspace
        max_n = min(30000, Data.Y_subspace.shape[0])
        D2 = euclidean_distances(Data.Y_subspace[:max_n], squared=True)
        med_sqdist = np.median(D2[np.triu_indices_from(D2, k=1)])
        self.args['sqdist'] = med_sqdist


        sigma_list = median_heuristic(self.args['sqdist'], beta=.5)
        self.sigma_var = torch.FloatTensor(sigma_list)

        # to get predictions

        ### experiment, might be that window = wnd_dim
        self.args['window'] = self.args['wnd_dim']
        self.window = self.args['window']
        self.num_workers = num_workers

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:

        X = batch[0].to(torch.float32)
        X_p, X_f = _history_future_separation(X, self.args['wnd_dim'])

        X_p = X_p.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])
        X_f = X_f.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])

        batch_size = X_p.size(0)

        X_p_enc, _ = self.netD(X_p)
        X_f_enc, _ = self.netD(X_f)

        Y_pred = batch_mmd2_loss(X_p_enc, X_f_enc, self.sigma_var.to(self.device))

        return Y_pred

    # Alternating schedule for optimizer steps (e.g. GANs)
    def optimizer_step(
            self,
            epoch: int,
            batch_idx: int,
            optimizer: torch.optim.Optimizer,
            optimizer_idx: int,
            optimizer_closure,
            on_tpu: bool = False,
            using_native_amp: bool = False,
            using_lbfgs: bool = False
    ):
        # update generator every CRITIC_ITERS steps
        if optimizer_idx == 0:
            if (batch_idx + 1) % self.args['CRITIC_ITERS'] == 0:
                # the closure (which includes the `training_step`) will be executed by `optimizer.step`
                optimizer.step(closure=optimizer_closure)
            else:
                # call the closure by itself to run `training_step` + `backward` without an optimizer step
                optimizer_closure()

        # update discriminator every step
        if optimizer_idx == 1:
            for p in self.netD.rnn_enc_layer.parameters():
                p.data.clamp_(-self.args['weight_clip'], self.args['weight_clip'])
            optimizer.step(closure=optimizer_closure)

    def training_step(self,
                      batch: torch.Tensor,
                      batch_idx: int,
                      optimizer_idx: int
                      ) -> torch.Tensor:

        # optimize discriminator (netD)
        if optimizer_idx == 1:
            X = batch[0].to(torch.float32)
            X_p, X_f = _history_future_separation(X, self.args['wnd_dim'])

            X_p = X_p.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])
            X_f = X_f.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])

            batch_size = X_p.size(0)

            # real data
            X_p_enc, X_p_dec = self.netD(X_p)
            X_f_enc, X_f_dec = self.netD(X_f)

            # fake data
            noise = torch.FloatTensor(1, batch_size, self.args['RNN_hid_dim']).normal_(0, 1)
            noise.requires_grad = False
            noise = noise.to(self.device)

            Y_f = self.netG(X_p, X_f, noise)
            Y_f = self.netG(X_p, X_f, noise)
            Y_f_enc, Y_f_dec = self.netD(Y_f)

            lossD, mmd2_real = mmdLossD(X_f, Y_f, X_f_enc, Y_f_enc, X_p_enc, X_f_dec, Y_f_dec,
                                        self.args['lambda_ae'], self.args['lambda_real'],
                                        self.sigma_var.to(self.device))
            lossD = (-1) * lossD
            self.log("train_loss_D", lossD, prog_bar=True)
            self.log("train_mmd2_real_D", mmd2_real, prog_bar=True)
            return lossD

        # optimize generator (netG)
        if optimizer_idx == 0:
            X = batch[0].to(torch.float32)
            X_p, X_f = _history_future_separation(X, self.args['wnd_dim'])

            X_p = X_p.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])
            X_f = X_f.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])

            batch_size = X_p.size(0)

            # real data
            X_f_enc, X_f_dec = self.netD(X_f)

            # fake data
            noise = torch.FloatTensor(1, batch_size, self.args['RNN_hid_dim']).normal_(0, 1)
            noise.requires_grad = False
            noise = noise.to(self.device)

            Y_f = self.netG(X_p, X_f, noise)
            Y_f_enc, Y_f_dec = self.netD(Y_f)

            # batchwise MMD2 loss between X_f and Y_f
            G_mmd2 = batch_mmd2_loss(X_f_enc, Y_f_enc, self.sigma_var.to(self.device))

            lossG = G_mmd2.mean()
            self.log("train_loss_G", lossG, prog_bar=True)

            return lossG

    def validation_step(self,
                        batch: torch.Tensor,
                        batch_idx: int
                        ) -> torch.Tensor:

        X = batch[0].to(torch.float32)
        X_p, X_f = _history_future_separation(X, self.args['wnd_dim'])

        X_p = X_p.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])
        X_f = X_f.reshape(-1, self.args['wnd_dim'], self.args['data_dim'])

        X_p_enc, _ = self.netD(X_p)
        X_f_enc, _ = self.netD(X_f)

        val_mmd2_real = batch_mmd2_loss(X_p_enc, X_f_enc, self.sigma_var.to(self.device))

        self.log('val_mmd2_real_D', val_mmd2_real, prog_bar=True)

        return val_mmd2_real

    def configure_optimizers(self) -> Tuple[torch.optim.Optimizer, torch.optim.Optimizer]:

        optimizerG = torch.optim.Adam(self.netG.parameters(),
                                      lr=self.args['lr'],
                                      weight_decay=self.args['weight_decay'])

        optimizerD = torch.optim.Adam(self.netD.parameters(),
                                      lr=self.args['lr'],
                                      weight_decay=self.args['weight_decay'])

        return optimizerG, optimizerD

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.args['batch_size'], shuffle=True,
                          num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.args['batch_size'], shuffle=False,
                          num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.args['batch_size'], shuffle=False,
                          num_workers=self.num_workers)

#### Experiment

In [ ]:
import scipy.io as sio

In [ ]:
Data.Y_subspace.shape

(1057, 3)

In [ ]:
data_try = sio.loadmat('/content/klcpd_code/data/beedance/beedance-1.mat')

In [ ]:
data_try

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Nov  1 15:32:43 2017',
 '__version__': '1.0',
 '__globals__': [],
 'Y': array([[0.34505421, 0.76464539, 0.54221329],
        [0.32433136, 0.77659554, 0.47098108],
        [0.30748942, 0.77960098, 0.4279893 ],
        ...,
        [0.6162483 , 0.21170825, 0.36854878],
        [0.6050455 , 0.21422565, 0.68399638],
        [0.62507353, 0.19284757, 0.30645727]]),
 'L': array([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])}

In [ ]:
Y = data_try['Y']
L = data_try['L']
T, D = Y.shape
trn_ratio = 0.6
val_ratio=0.8
n_trn = int(np.ceil(T * trn_ratio))
p_wnd_dim = 25 
trn_set_idx = range(p_wnd_dim, n_trn)
n_val = int(np.ceil(T * val_ratio))

val_set_idx = range(n_trn, n_val)
tst_set_idx = range(n_val, T)

In [ ]:
trn_set_idx

range(25, 635)

In [ ]:
Y_train = Y[trn_set_idx]
L_train = L[trn_set_idx]
Y_val = Y[val_set_idx]
L_val = L[val_set_idx]
Y_tst = Y[tst_set_idx]
L_tst = L[tst_set_idx]

In [ ]:
torch.tensor(Y_train).size()

torch.Size([610, 3])

In [ ]:
torch.tensor(L_train).size()

torch.Size([610, 1])

In [ ]:
L_train.reshape(-1, ).shape

(610,)

In [ ]:
Train_try = TensorDataset(torch.tensor(Y_train), torch.tensor(L_train))
Test_try = TensorDataset(torch.tensor(Y_tst), torch.tensor(L_tst))
Val_try = TensorDataset(torch.tensor(Y_val), torch.tensor(L_val))

In [ ]:
def load_data(self, trn_ratio=0.6, val_ratio=0.8):
    dataset = sio.loadmat(self.data_path)
    self.Y = dataset['Y']                                   # Y: time series data, time length x number of variables
    self.L = dataset['L']                                   # L: label of anomaly, time length x 1
    self.T, self.D = self.Y.shape                           # T: time length; D: variable dimension
    self.n_trn = int(np.ceil(self.T * trn_ratio))           # n_trn: first index of val set
    self.n_val = int(np.ceil(self.T * val_ratio))           # n_val: first index of tst set
    self.var_dim = self.D * self.sub_dim

def split_data(self):
    self.p_wnd_dim = 25
    trn_set_idx = range(self.p_wnd_dim, self.n_trn)
    val_set_idx = range(self.n_trn, self.n_val)
    tst_set_idx = range(self.n_val, self.T)
    print('n_trn ', len(trn_set_idx), 'n_val ', len(val_set_idx), 'n_tst ', len(tst_set_idx))
    self.trn_set = self.__batchify(trn_set_idx)
    self.val_set = self.__batchify(val_set_idx)
    self.tst_set = self.__batchify(tst_set_idx)

In [ ]:
import os
import numpy as np
import scipy.io as sio
import math
import torch
from torch.autograd import Variable


class DataLoader(object):
    def __init__(self, args, trn_ratio=0.6, val_ratio=0.8):
        self.cuda = args.cuda
        self.data_path = args.data_path
        self.p_wnd_dim = 25
        self.f_wnd_dim = args.wnd_dim
        self.sub_dim = args.sub_dim
        self.batch_size = args.batch_size

        # load data
        self.load_data(trn_ratio=trn_ratio, val_ratio=val_ratio)

        # prepare data
        self.prepare_data()

        # split data into trn/val/tst set
        self.split_data()

    # load data
    def load_data(self, trn_ratio=0.6, val_ratio=0.8):
        assert(os.path.lexists(self.data_path))
        dataset = sio.loadmat(self.data_path)
        self.Y = dataset['Y']                                   # Y: time series data, time length x number of variables
        self.L = dataset['L']                                   # L: label of anomaly, time length x 1
        self.T, self.D = self.Y.shape                           # T: time length; D: variable dimension
        self.n_trn = int(np.ceil(self.T * trn_ratio))           # n_trn: first index of val set
        self.n_val = int(np.ceil(self.T * val_ratio))           # n_val: first index of tst set
        self.var_dim = self.D * self.sub_dim

    # prepare subspace data (Hankel matrix)
    def prepare_data(self):
        # T x D x sub_dim
        self.Y_subspace = np.zeros((self.T, self.D, self.sub_dim))
        for t in range(self.sub_dim, self.T):
            for d in range(self.D):
                self.Y_subspace[t, d, :] = self.Y[t-self.sub_dim+1:t+1, d].flatten()

        # Y_subspace is now T x (Dxsub_dim)
        self.Y_subspace = self.Y_subspace.reshape(self.T, -1)

    # split data into trn/val/tst set
    def split_data(self):
        trn_set_idx = range(self.p_wnd_dim, self.n_trn)
        val_set_idx = range(self.n_trn, self.n_val)
        tst_set_idx = range(self.n_val, self.T)
        print('n_trn ', len(trn_set_idx), 'n_val ', len(val_set_idx), 'n_tst ', len(tst_set_idx))
        self.trn_set = self.__batchify(trn_set_idx)
        self.val_set = self.__batchify(val_set_idx)
        self.tst_set = self.__batchify(tst_set_idx)

    # convert augmented data in Hankel matrix to origin time series
    # input: X_f, whose shape is batch_size x seq_len x (D*sub_dim)
    # output: Y_t, whose shape is batch_size x D
    def repack_data(self, X_f, batch_size):
        Y_t = X_f[:, 0, :].contiguous().view(batch_size, self.D, self.sub_dim)
        return Y_t[:, :, -1]

    def __batchify(self, idx_set):
        n = len(idx_set)
        L = torch.zeros((n, 1))                             # anomaly label
        Y = torch.zeros((n, self.D))                        # true signal
        X_p = torch.zeros((n, self.p_wnd_dim, self.var_dim))  # past window buffer
        X_f = torch.zeros((n, self.f_wnd_dim, self.var_dim))  # future window buffer

        # XXX: dirty trick to augment the last buffer
        data = np.concatenate((self.Y_subspace, self.Y_subspace[-self.f_wnd_dim:, :]))
        for i in range(n):
            l = idx_set[i] - self.p_wnd_dim
            m = idx_set[i]
            u = idx_set[i] + self.f_wnd_dim
            X_p[i, :, :] = torch.from_numpy(data[l:m, :])
            X_f[i, :, :] = torch.from_numpy(data[m:u, :])
            Y[i, :] = torch.from_numpy(self.Y[m, :])
            L[i] = torch.from_numpy(self.L[m])
        return {'X_p': X_p, 'X_f': X_f, 'Y': Y, 'L': L}

    def get_batches(self, data_set, batch_size, shuffle=False):
        X_p, X_f = data_set['X_p'], data_set['X_f']
        Y, L = data_set['Y'], data_set['L']
        length = len(Y)
        if shuffle:
            index = torch.randperm(length)
        else:
            index = torch.LongTensor(range(length))
        s_idx = 0
        while (s_idx < length):
            e_idx = min(length, s_idx + batch_size)
            excerpt = index[s_idx:e_idx]
            X_p_batch, X_f_batch = X_p[excerpt], X_f[excerpt]
            Y_batch, L_batch = Y[excerpt], L[excerpt]
            if self.cuda:
                X_p_batch = X_p_batch.cuda()
                X_f_batch = X_f_batch.cuda()
                Y_batch = Y_batch.cuda()
                L_batch = L_batch.cuda()

            data = [Variable(X_p_batch),
                    Variable(X_f_batch),
                    Variable(Y_batch),
                    Variable(L_batch)]
            yield data
            s_idx += batch_size

In [ ]:
type(Data.trn_set)

dict

In [ ]:
Data.D

3

In [ ]:
Train.keys()

dict_keys(['X_p', 'X_f', 'Y', 'L'])

In [ ]:
Train['X_p'].shape

torch.Size([610, 25, 3])

In [ ]:
Train['X_f'].shape

torch.Size([610, 10, 3])

In [ ]:
Train['Y'].shape

torch.Size([610, 3])

In [ ]:
Train['L'].shape

torch.Size([610, 1])

In [ ]:
Test['L'].shape

torch.Size([211, 1])

In [ ]:
d

{'data_path': '//content/klcpd_code/data/beedance/beedance-1.mat',
 'trn_ratio': 0.6,
 'val_ratio': 0.8,
 'gpu': 0,
 'cuda': True,
 'random_seed': 1126,
 'wnd_dim': 10,
 'sub_dim': 1,
 'RNN_hid_dim': 10,
 'batch_size': 128,
 'max_iter': 100,
 'optim': 'adam',
 'lr': 0.0003,
 'weight_decay': 0.0,
 'momentum': 0.0,
 'grad_clip': 10.0,
 'eval_freq': 50,
 'CRITIC_ITERS': 5,
 'weight_clip': 0.1,
 'lambda_ae': 0.001,
 'lambda_real': 0.1,
 'save_path': '/content/exp_simulate/jumpingmean/save_RNN'}

In [ ]:
dataroot = '/content/klcpd_code/data'
dataset = 'beedance'
gpu = 0
wnd_dim_list = [5, 10, 15, 20, 25, 30]
lambda_ae = 1e-3
lambda_real = 1
max_iter = 2000
batch_size = 64
eval_freq = 25
weight_clip = .1
#parser.add_argument('--save_dir', type=str, default='experiment_log', help='experiment directory for saving train log and models')
save_dir = 'experiment_log'

In [ ]:
def run_klcpd_exp(dataset):
    for wnd_dim in wnd_dim_list:
        data_dir = os.path.join(dataroot, dataset)
        for data_path in glob.glob('%s/*.mat' % (data_dir)):
            data_name = data_path.split('/')[-1].split('.')[0]
            save_name = '%s.wnd-%d.lambda_ae-%f.lambda_real-%f.clip-%f' % (data_name, wnd_dim, lambda_ae, lambda_real, weight_clip)
            #save_path = '%s/%s' % (args.save_dir, save_name)
            save_path = '%s/%s' % (save_dir, save_name)
            trn_log_path = '%s.trn.log' % (save_path)
            option = '--data_path %s --wnd_dim %d --lambda_ae %f --lambda_real %f --weight_clip %f --max_iter %d --batch_size %d --eval_freq %d --save_path %s' \
                % (data_path, wnd_dim, lambda_ae, lambda_real, weight_clip, max_iter, batch_size, eval_freq, save_path)
            cmd = 'CUDA_VISIBLE_DEVICES=%d python -u klcpd.py %s 2>&1 | tee %s' % (gpu, option, trn_log_path)
            #print(cmd)
            os.system(cmd)

#### Initialize parameters

In [ ]:
from __future__ import print_function
import argparse
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

def median_heuristic(X, beta=0.5):
    max_n = min(30000, X.shape[0])
    D2 = euclidean_distances(X[:max_n], squared=True)
    med_sqdist = np.median(D2[np.triu_indices_from(D2, k=1)])
    beta_list = [beta**2, beta**1, 1, (1.0/beta)**1, (1.0/beta)**2]
    return [med_sqdist * b for b in beta_list]


In [ ]:

# ========= Setup input argument =========#
parser = argparse.ArgumentParser(description='PyTorch Time series forecasting')
#parser.add_argument('--data_path', type=str, required=True, help='path to data in matlab format')
parser.add_argument('--data_path', type=str, default = '//content/klcpd_code/data/beedance/beedance-1.mat', help='path to data in matlab format')
parser.add_argument('--trn_ratio', type=float, default=0.6,help='how much data used for training')
parser.add_argument('--val_ratio', type=float, default=0.8,help='how much data used for validation')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--cuda', type=str, default=True, help='use gpu or not')
parser.add_argument('--random_seed', type=int, default=1126,help='random seed')
#parser.add_argument('--wnd_dim', type=int, required=True, default=10, help='window size (past and future)')
parser.add_argument('--wnd_dim', type=int, default=10, help='window size (past and future)')
parser.add_argument('--sub_dim', type=int, default=1, help='dimension of subspace embedding')

# RNN hyperparemters
parser.add_argument('--RNN_hid_dim', type=int, default=10, help='number of RNN hidden units')

# optimization
parser.add_argument('--batch_size', type=int, default=128, help='batch size for training')
parser.add_argument('--max_iter', type=int, default=100, help='max iteration for pretraining RNN')
parser.add_argument('--optim', type=str, default='adam', help='sgd|rmsprop|adam for optimization method')
parser.add_argument('--lr', type=float, default=3e-4, help='learning rate')
parser.add_argument('--weight_decay', type=float, default=0., help='weight decay (L2 regularization)')
parser.add_argument('--momentum', type=float, default=0.0, help='momentum for sgd')
parser.add_argument('--grad_clip', type=float, default=10.0, help='gradient clipping for RNN (both netG and netD)')
parser.add_argument('--eval_freq', type=int, default=50, help='evaluation frequency per generator update')

# GAN
parser.add_argument('--CRITIC_ITERS', type=int, default=5, help='number of updates for critic per generator')
parser.add_argument('--weight_clip', type=float, default=.1, help='weight clipping for crtic')
parser.add_argument('--lambda_ae', type=float, default=0.001, help='coefficient for the reconstruction loss')
parser.add_argument('--lambda_real', type=float, default=0.1, help='coefficient for the real MMD2 loss')

# save models  /content/exp_simulate/jumpingmean/save_RNN
parser.add_argument('--save_path', type=str,  default='/content/exp_simulate/jumpingmean/save_RNN',help='path to save the final model')

args, unknown = parser.parse_known_args()

In [ ]:
!mkdir 'exp_simulate'
!cd '/content/exp_simulate'

mkdir: cannot create directory ‘exp_simulate’: File exists


In [ ]:
!cd /content/exp_simulate
!pwd

/content


In [ ]:
if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)
assert(os.path.isdir(args.save_path))
# assert(args.sub_dim == 1)

#XXX For Yahoo dataset, trn_ratio=0.50, val_ratio=0.75
if 'yahoo' in args.data_path:
    args.trn_ratio = 0.50
    args.val_ratio = 0.75

In [ ]:
d = vars(args)

In [ ]:
if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)
assert(os.path.isdir(args.save_path))
# assert(args.sub_dim == 1)

#XXX For Yahoo dataset, trn_ratio=0.50, val_ratio=0.75
if 'yahoo' in args.data_path:
    args.trn_ratio = 0.50
    args.val_ratio = 0.75



# ========= Setup GPU device and fix random seed=========#
if torch.cuda.is_available():
    args.cuda = True
    torch.cuda.set_device(args.gpu)
    print('Using GPU device', torch.cuda.current_device())
else:
    raise EnvironmentError("GPU device not available!")
np.random.seed(seed=args.random_seed)
random.seed(args.random_seed)
torch.manual_seed(args.random_seed)
torch.cuda.manual_seed(args.random_seed)
# [INFO] cudnn.benckmark=True enable cudnn auto-tuner to find the best algorithm to use for your hardware
# [INFO] benchmark mode is good whenever input sizes of network do not vary much!!!
# [INFO] https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
# [INFO] https://discuss.pytorch.org/t/pytorch-performance/3079/2
torch.backends.cudnn.benchmark == True

# [INFO} For reproducibility and debugging, set cudnn.enabled=False
# [INFO] Some operations are non-deterministic when cudnn.enabled=True
# [INFO] https://discuss.pytorch.org/t/non-determinisic-results/459
# [INFO] https://discuss.pytorch.org/t/non-reproducible-result-with-gpu/1831
torch.backends.cudnn.enabled = True


# ========= Load Dataset and initialize model=========#
Data = DataLoader(args, trn_ratio=args.trn_ratio, val_ratio=args.val_ratio)
netG = NetG(args, Data)
netD = NetD(args, Data)

Using GPU device 0
n_trn  610 n_val  211 n_tst  211


In [ ]:
if args.cuda:
    netG.cuda()
    netD.cuda()
netG_params_count = sum([p.nelement() for p in netG.parameters()])
netD_params_count = sum([p.nelement() for p in netD.parameters()])
print(netG)
print(netD)
print('netG has number of parameters: %d' % (netG_params_count))
print('netD has number of parameters: %d' % (netD_params_count))
one = torch.cuda.FloatTensor([1])
one = torch.tensor(1, dtype=torch.float).cuda()
mone = one * -1

NetG(
  (rnn_enc_layer): GRU(3, 10, batch_first=True)
  (rnn_dec_layer): GRU(3, 10, batch_first=True)
  (fc_layer): Linear(in_features=10, out_features=3, bias=True)
)
NetD(
  (rnn_enc_layer): GRU(3, 10, batch_first=True)
  (rnn_dec_layer): GRU(10, 3, batch_first=True)
)
netG has number of parameters: 933
netD has number of parameters: 585


In [ ]:
one = torch.tensor(1, dtype=torch.float).cuda()
one

tensor(1., device='cuda:0')

In [ ]:
mone

tensor([-1.], device='cuda:0')

#### Optimizers

In [ ]:
import torch.optim as optim

class Optim(object):

    def _makeOptimizer(self):
        if self.method == 'sgd':
            self.optimizer = optim.SGD(self.params, lr=self.lr, weight_decay=self.weight_decay, momentum=self.momentum)
        elif self.method == 'adagrad':
            self.optimizer = optim.Adagrad(self.params, lr=self.lr, weight_decay=self.weight_decay)
        elif self.method == 'rmsprop':
            self.optimizer = optim.RMSprop(self.params, lr=self.lr, weight_decay=self.weight_decay, momentum=self.momentum)
        elif self.method == 'adam':
            self.optimizer = optim.Adam(self.params, lr=self.lr, weight_decay=self.weight_decay)
        else:
            raise RuntimeError("Invalid optim method: " + self.method)

    def __init__(self, params, method, lr=0.1, grad_clip=10.0, weight_decay=0., momentum=0.9):
        self.params = list(params)
        self.lr = lr
        self.max_norm = grad_clip
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.method = method
        self._makeOptimizer()

    def zero_grad(self):
        self.optimizer.zero_grad();

    def step(self):
        # Compute gradients norm.
        total_norm = 0
        for p in self.params:
            total_norm += p.grad.data.norm(2) ** 2
        total_norm = total_norm ** (1. / 2)
        clip_coef = self.max_norm / (total_norm + 1e-6)

        # grading clipping
        if clip_coef < 1:
            for p in self.params:
                p.grad.data.mul_(clip_coef)
        self.optimizer.step()

    # decay learning rate if val perf does not improve or we hit the start_decay_at limit
    def updateLearningRate(self, ppl, epoch):
        if self.start_decay_at is not None and epoch >= self.start_decay_at:
            self.start_decay = True
        if self.last_ppl is not None and ppl > self.last_ppl:
            self.start_decay = True

        if self.start_decay:
            self.lr = self.lr * self.lr_decay
            print("Decaying learning rate to %g" % self.lr)
        #only decay for one epoch
        self.start_decay = False

        self.last_ppl = ppl
        self._makeOptimizer()


        # ========= Setup loss function and optimizer  =========#
optimizerG = Optim(netG.parameters(),
                   args.optim,
                   lr=args.lr,
                   grad_clip=args.grad_clip,
                   weight_decay=args.weight_decay,
                   momentum=args.momentum)

optimizerD = Optim(netD.parameters(),
                   args.optim,
                   lr=args.lr,
                   grad_clip=args.grad_clip,
                   weight_decay=args.weight_decay,
                   momentum=args.momentum)


#### Data Inquiry

In [ ]:
Train = Data.trn_set

In [ ]:
Test = Data.tst_set

In [ ]:
Train

In [ ]:
d

{'data_path': '//content/klcpd_code/data/beedance/beedance-1.mat',
 'trn_ratio': 0.6,
 'val_ratio': 0.8,
 'gpu': 0,
 'cuda': True,
 'random_seed': 1126,
 'wnd_dim': 10,
 'sub_dim': 1,
 'RNN_hid_dim': 10,
 'batch_size': 128,
 'max_iter': 100,
 'optim': 'adam',
 'lr': 0.0003,
 'weight_decay': 0.0,
 'momentum': 0.0,
 'grad_clip': 10.0,
 'eval_freq': 50,
 'CRITIC_ITERS': 5,
 'weight_clip': 0.1,
 'lambda_ae': 0.001,
 'lambda_real': 0.1,
 'save_path': '/content/exp_simulate/jumpingmean/save_RNN',
 'sqdist': 0.15892872017303572}

In [ ]:
net = KLCPD(netG, netD, d, Train_try, Test_try)
trainer = pl.Trainer()

IndexError: ignored

In [ ]:


trainer.fit(net)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type | Params
------------------------------
0 | netG | NetG | 933   
1 | netD | NetD | 585   
------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

IndexError: ignored

In [ ]:
experiment.configure_optimizers()

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: False
     lr: 0.0003
     maximize: False
     weight_decay: 0.0
 ), Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: False
     lr: 0.0003
     maximize: False
     weight_decay: 0.0
 ))

In [ ]:
experiment.train_dataloader()

In [ ]:
def median_heuristic(X, beta=0.5):
    max_n = min(30000, X.shape[0])
    D2 = euclidean_distances(X[:max_n], squared=True)
    med_sqdist = np.median(D2[np.triu_indices_from(D2, k=1)])
    beta_list = [beta**2, beta**1, 1, (1.0/beta)**1, (1.0/beta)**2]
    return [med_sqdist * b for b in beta_list]

In [ ]:
# sigma for mixture of RBF kernel in MMD
#sigma_list = [1.0]
#sigma_list = mmd_util.median_heuristic(Data.Y_subspace, beta=1.)
sigma_list = median_heuristic(Data.Y_subspace, beta=.5)
sigma_var = torch.FloatTensor(sigma_list).cuda()
print('sigma_list:', sigma_var)


sigma_list: tensor([0.0397, 0.0795, 0.1589, 0.3179, 0.6357], device='cuda:0')


In [ ]:

# ========= Main loop for adversarial training kernel with negative samples X_f + noise =========#
Y_val = Data.val_set['Y'].numpy()
L_val = Data.val_set['L'].numpy()
Y_tst = Data.tst_set['Y'].numpy()
L_tst = Data.tst_set['L'].numpy()

n_batchs = int(math.ceil(len(Data.trn_set['Y']) / float(args.batch_size)))
print('n_batchs', n_batchs, 'batch_size', args.batch_size)

n_batchs 5 batch_size 128


In [ ]:
args.max_iter

100

In [ ]:
n_batchs

5

In [ ]:
args.CRITIC_ITERS

5

In [ ]:
bidx == n_batchs

True

In [ ]:
bidx

5

In [ ]:
        trn_loader = Data.get_batches(Data.trn_set, batch_size=args.batch_size, shuffle=True)
        bidx = 0
        print(bidx)
        ############################
        # (1) Update D network
        ############################
        for p in netD.parameters():
            p.requires_grad = True

        for diters in range(args.CRITIC_ITERS):
            # clamp parameters of NetD encoder to a cube
            for p in netD.rnn_enc_layer.parameters():
                p.data.clamp_(-args.weight_clip, args.weight_clip)
            if bidx == n_batchs:
                break

            inputs = next(trn_loader)
            X_p, X_f, Y_true = inputs[0], inputs[1], inputs[2]
            batch_size = X_p.size(0)
            bidx += 1

            # real data
            X_p_enc, X_p_dec = netD(X_p)
            X_f_enc, X_f_dec = netD(X_f)

            # fake data
            noise = torch.cuda.FloatTensor(1, batch_size, args.RNN_hid_dim).normal_(0, 1)
            noise = Variable(noise, volatile=True) # total freeze netG
            Y_f = Variable(netG(X_p, X_f, noise).data)
            Y_f_enc, Y_f_dec = netD(Y_f)

            # batchwise MMD2 loss between X_f and Y_f
            D_mmd2 = batch_mmd2_loss(X_f_enc, Y_f_enc, sigma_var)

            # batchwise MMD loss between X_p and X_f
            mmd2_real = batch_mmd2_loss(X_p_enc, X_f_enc, sigma_var)

            # reconstruction loss
            real_L2_loss = torch.mean((X_f - X_f_dec)**2)
            #real_L2_loss = torch.mean((X_p - X_p_dec)**2)
            fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2)
            #fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2) * 0.0

            # update netD
            netD.zero_grad()
            lossD = D_mmd2.mean() - lambda_ae * (real_L2_loss + fake_L2_loss) - lambda_real * mmd2_real.mean()
            #lossD = 0.0 * D_mmd2.mean() - lambda_ae * (real_L2_loss + fake_L2_loss) - lambda_real * mmd2_real.mean()
            #lossD = -real_L2_loss
            #lossD.backward(mone)
            lossD.backward(mone)
            optimizerD.step()

0


<ipython-input-71-1e47c6509d7c>:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  noise = Variable(noise, volatile=True) # total freeze netG


In [ ]:
batch_mmd2_loss(X_p_enc, X_f_enc, sigma_var).shape

torch.Size([98])

In [ ]:
real_L2_loss.shape

torch.Size([])

In [ ]:
lambda_ae * (real_L2_loss + fake_L2_loss)

tensor(0.0003, device='cuda:0', grad_fn=<MulBackward0>)

In [ ]:
X_f_enc.shape, Y_f_enc.shape

(torch.Size([98, 10, 10]), torch.Size([98, 10, 10]))

In [ ]:
lossD

tensor(3.7536, device='cuda:0', grad_fn=<SubBackward0>)

In [ ]:
one

tensor(1., device='cuda:0')

In [ ]:
import time


lambda_ae = args.lambda_ae
lambda_real = args.lambda_real
gen_iterations = 0
total_time = 0.
best_epoch = -1
best_val_mae = 1e+6
best_val_auc = -1
best_tst_auc = -1
best_mmd_real = 1e+6
start_time = time.time()
print('start training: lambda_ae', lambda_ae, 'lambda_real', lambda_real, 'weight_clip', args.weight_clip)
for epoch in range(1, args.max_iter + 1):
    trn_loader = Data.get_batches(Data.trn_set, batch_size=args.batch_size, shuffle=True)
    bidx = 0
    while bidx < n_batchs:
        #print(bidx)
        ############################
        # (1) Update D network
        ############################
        for p in netD.parameters():
            p.requires_grad = True

        for diters in range(args.CRITIC_ITERS):
            # clamp parameters of NetD encoder to a cube
            for p in netD.rnn_enc_layer.parameters():
                p.data.clamp_(-args.weight_clip, args.weight_clip)
            if bidx == n_batchs:
                break

            inputs = next(trn_loader)
            X_p, X_f, Y_true = inputs[0], inputs[1], inputs[2]
            batch_size = X_p.size(0)
            bidx += 1

            # real data
            X_p_enc, X_p_dec = netD(X_p)
            X_f_enc, X_f_dec = netD(X_f)

            # fake data
            noise = torch.cuda.FloatTensor(1, batch_size, args.RNN_hid_dim).normal_(0, 1)
            noise = Variable(noise, volatile=True) # total freeze netG
            Y_f = Variable(netG(X_p, X_f, noise).data)
            Y_f_enc, Y_f_dec = netD(Y_f)

            # batchwise MMD2 loss between X_f and Y_f
            D_mmd2 = batch_mmd2_loss(X_f_enc, Y_f_enc, sigma_var)

            # batchwise MMD loss between X_p and X_f
            mmd2_real = batch_mmd2_loss(X_p_enc, X_f_enc, sigma_var)

            # reconstruction loss
            real_L2_loss = torch.mean((X_f - X_f_dec)**2)
            #real_L2_loss = torch.mean((X_p - X_p_dec)**2)
            fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2)
            #fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2) * 0.0

            # update netD
            netD.zero_grad()
            lossD = D_mmd2.mean() - lambda_ae * (real_L2_loss + fake_L2_loss) - lambda_real * mmd2_real.mean()
            #lossD = 0.0 * D_mmd2.mean() - lambda_ae * (real_L2_loss + fake_L2_loss) - lambda_real * mmd2_real.mean()
            #lossD = -real_L2_loss
            #lossD.backward(mone)
            lossD.backward(mone)
            optimizerD.step()

        ############################
        # (2) Update G network
        ############################
        for p in netD.parameters():
            p.requires_grad = False  # to avoid computation

        if bidx == n_batchs:
            break

        inputs = next(trn_loader)
        X_p, X_f = inputs[0], inputs[1]
        batch_size = X_p.size(0)
        bidx += 1

        # real data
        X_f_enc, X_f_dec = netD(X_f)

        # fake data
        noise = torch.cuda.FloatTensor(1, batch_size, args.RNN_hid_dim).normal_(0, 1)
        noise = Variable(noise)
        Y_f = netG(X_p, X_f, noise)
        Y_f_enc, Y_f_dec = netD(Y_f)

        # batchwise MMD2 loss between X_f and Y_f
        G_mmd2 = batch_mmd2_loss(X_f_enc, Y_f_enc, sigma_var)

        # update netG
        netG.zero_grad()
        lossG = G_mmd2.mean()
        #lossG = 0.0 * G_mmd2.mean()
        lossG.backward(one)
        optimizerG.step()

        #G_mmd2 = Variable(torch.FloatTensor(batch_size).zero_())
        gen_iterations += 1

        print('[%5d/%5d] [%5d/%5d] [%6d] D_mmd2 %.4e G_mmd2 %.4e mmd2_real %.4e real_L2 %.6f fake_L2 %.6f'
              % (epoch, args.max_iter, bidx, n_batchs, gen_iterations,
                 D_mmd2.mean().data[0], G_mmd2.mean().data[0], mmd2_real.mean().data[0],
                 real_L2_loss.data[0], fake_L2_loss.data[0]))

        if gen_iterations % args.eval_freq == 0:
            # ========= Main block for evaluate MMD(X_p_enc, X_f_enc) on RNN codespace  =========#
            val_dict = valid_epoch(Data, Data.val_set, netD, args.batch_size, Y_val, L_val)
            tst_dict = valid_epoch(Data, Data.tst_set, netD, args.batch_size, Y_tst, L_tst)
            total_time = time.time() - start_time
            print('iter %4d tm %4.2fm val_mse %.1f val_mae %.1f val_auc %.6f'
                    % (epoch, total_time / 60.0, val_dict['mse'], val_dict['mae'], val_dict['auc']), end='')

            print (" tst_mse %.1f tst_mae %.1f tst_auc %.6f" % (tst_dict['mse'], tst_dict['mae'], tst_dict['auc']), end='')

            assert(np.isnan(val_dict['auc']) != True)
            #if val_dict['auc'] > best_val_auc:
            #if val_dict['auc'] > best_val_auc and mmd2_real.mean().data[0] < best_mmd_real:
            if mmd2_real.mean().data[0] < best_mmd_real:
                best_mmd_real = mmd2_real.mean().data[0]
                best_val_mae = val_dict['mae']
                best_val_auc = val_dict['auc']
                best_tst_auc = tst_dict['auc']
                best_epoch = epoch
                save_pred_name = '%s/pred.pkl' % (args.save_path)
                with open(save_pred_name, 'wb') as f:
                    pickle.dump(tst_dict, f)
                torch.save(netG.state_dict(), '%s/netG.pkl' % (args.save_path))
                torch.save(netD.state_dict(), '%s/netD.pkl' % (args.save_path))
            print(" [best_val_auc %.6f best_tst_auc %.6f best_epoch %3d]" % (best_val_auc, best_tst_auc, best_epoch))

        # stopping condition
        #if best_mmd_real < 1e-4:
        if mmd2_real.mean().data[0] < 1e-5:
            exit(0)

start training: lambda_ae 0.001 lambda_real 0.1 weight_clip 0.1


<ipython-input-81-f6d3bbf23736>:44: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  noise = Variable(noise, volatile=True) # total freeze netG


In [ ]:
print(" [best_val_auc %.6f best_tst_auc %.6f best_epoch %3d]" % (best_val_auc, best_tst_auc, best_epoch))

 [best_val_auc -1.000000 best_tst_auc -1.000000 best_epoch  -1]


In [ ]:
def median_heuristic(X, beta=0.5):
    max_n = min(30000, X.shape[0])
    D2 = euclidean_distances(X[:max_n], squared=True)
    med_sqdist = np.median(D2[np.triu_indices_from(D2, k=1)])
    beta_list = [beta**2, beta**1, 1, (1.0/beta)**1, (1.0/beta)**2]
    return [med_sqdist * b for b in beta_list]


# X_p_enc: batch_size x seq_len x hid_dim
# X_f_enc: batch_size x seq_len x hid_dim
# hid_dim could be either dataspace_dim or codespace_dim
# return: MMD2(X_p_enc[i,:,:], X_f_enc[i,:,:]) for i = 1:batch_size
def batch_mmd2_loss(X_p_enc, X_f_enc, sigma_var):
    # some constants
    n_basis = 1024
    gumbel_lmd = 1e+6
    cnst = math.sqrt(1. / n_basis)
    n_mixtures = sigma_var.size(0)
    n_samples = n_basis * n_mixtures
    batch_size, seq_len, nz = X_p_enc.size()

    # gumbel trick to get masking matrix to uniformly sample sigma
    # input: (batch_size*n_samples, nz)
    # output: (batch_size, n_samples, nz)
    def sample_gmm(W, batch_size):
        U = torch.cuda.FloatTensor(batch_size*n_samples, n_mixtures).uniform_()
        sigma_samples = F.softmax(U * gumbel_lmd).matmul(sigma_var)
        W_gmm = W.mul(1. / sigma_samples.unsqueeze(1))
        W_gmm = W_gmm.view(batch_size, n_samples, nz)
        return W_gmm

    W = Variable(torch.cuda.FloatTensor(batch_size*n_samples, nz).normal_(0, 1))
    W_gmm = sample_gmm(W, batch_size)                                   # batch_size x n_samples x nz
    W_gmm = torch.transpose(W_gmm, 1, 2).contiguous()                   # batch_size x nz x n_samples
    XW_p = torch.bmm(X_p_enc, W_gmm)                                    # batch_size x seq_len x n_samples
    XW_f = torch.bmm(X_f_enc, W_gmm)                                    # batch_size x seq_len x n_samples
    z_XW_p = cnst * torch.cat((torch.cos(XW_p), torch.sin(XW_p)), 2)
    z_XW_f = cnst * torch.cat((torch.cos(XW_f), torch.sin(XW_f)), 2)
    batch_mmd2_rff = torch.sum((z_XW_p.mean(1) - z_XW_f.mean(1))**2, 1)
    return batch_mmd2_rff

In [ ]:
# ========= Main loop for adversarial training kernel with negative samples X_f + noise =========#
Y_val = Data.val_set['Y'].numpy()
L_val = Data.val_set['L'].numpy()
Y_tst = Data.tst_set['Y'].numpy()
L_tst = Data.tst_set['L'].numpy()

n_batchs = int(math.ceil(len(Data.trn_set['Y']) / float(args.batch_size)))
print('n_batchs', n_batchs, 'batch_size', args.batch_size)

lambda_ae = args.lambda_ae
lambda_real = args.lambda_real
gen_iterations = 0
total_time = 0.
best_epoch = -1
best_val_mae = 1e+6
best_val_auc = -1
best_tst_auc = -1
best_mmd_real = 1e+6
start_time = time.time()
print('start training: lambda_ae', lambda_ae, 'lambda_real', lambda_real, 'weight_clip', args.weight_clip)
for epoch in range(1, args.max_iter + 1):
    trn_loader = Data.get_batches(Data.trn_set, batch_size=args.batch_size, shuffle=True)
    bidx = 0
    while bidx < n_batchs:
        ############################
        # (1) Update D network
        ############################
        for p in netD.parameters():
            p.requires_grad = True

        for diters in range(args.CRITIC_ITERS):
            # clamp parameters of NetD encoder to a cube
            for p in netD.rnn_enc_layer.parameters():
                p.data.clamp_(-args.weight_clip, args.weight_clip)
            if bidx == n_batchs:
                break

            inputs = next(trn_loader)
            X_p, X_f, Y_true = inputs[0], inputs[1], inputs[2]
            batch_size = X_p.size(0)
            bidx += 1

            # real data
            X_p_enc, X_p_dec = netD(X_p)
            X_f_enc, X_f_dec = netD(X_f)

            # fake data
            noise = torch.cuda.FloatTensor(1, batch_size, args.RNN_hid_dim).normal_(0, 1)
            noise = Variable(noise, volatile=True) # total freeze netG
            Y_f = Variable(netG(X_p, X_f, noise).data)
            Y_f_enc, Y_f_dec = netD(Y_f)

            # batchwise MMD2 loss between X_f and Y_f
            D_mmd2 = batch_mmd2_loss(X_f_enc, Y_f_enc, sigma_var)

            # batchwise MMD loss between X_p and X_f
            mmd2_real = batch_mmd2_loss(X_p_enc, X_f_enc, sigma_var)

            # reconstruction loss
            real_L2_loss = torch.mean((X_f - X_f_dec)**2)
            #real_L2_loss = torch.mean((X_p - X_p_dec)**2)
            fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2)
            #fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2) * 0.0

            # update netD
            netD.zero_grad()
            lossD = D_mmd2.mean() - lambda_ae * (real_L2_loss + fake_L2_loss) - lambda_real * mmd2_real.mean()
            #lossD = 0.0 * D_mmd2.mean() - lambda_ae * (real_L2_loss + fake_L2_loss) - lambda_real * mmd2_real.mean()
            #lossD = -real_L2_loss
            lossD.backward(mone)
            optimizerD.step()

        ############################
        # (2) Update G network
        ############################

        

        for p in netD.parameters():
            p.requires_grad = False  # to avoid computation

        #if bidx == n_batchs:
        #    break
        print('G network')  
        #inputs = next(trn_loader)
        X_p, X_f = inputs[0], inputs[1]
        batch_size = X_p.size(0)
        bidx += 1

        # real data
        X_f_enc, X_f_dec = netD(X_f)

        # fake data
        noise = torch.cuda.FloatTensor(1, batch_size, args.RNN_hid_dim).normal_(0, 1)
        noise = Variable(noise)
        Y_f = netG(X_p, X_f, noise)
        Y_f_enc, Y_f_dec = netD(Y_f)

        # batchwise MMD2 loss between X_f and Y_f
        G_mmd2 = batch_mmd2_loss(X_f_enc, Y_f_enc, sigma_var)

        # update netG
        netG.zero_grad()
        lossG = G_mmd2.mean()
        #lossG = 0.0 * G_mmd2.mean()
        lossG.backward(one)
        optimizerG.step()

        #G_mmd2 = Variable(torch.FloatTensor(batch_size).zero_())
        gen_iterations += 1

        print('[%5d/%5d] [%5d/%5d] [%6d] D_mmd2 %.4e G_mmd2 %.4e mmd2_real %.4e real_L2 %.6f fake_L2 %.6f'
              % (epoch, args.max_iter, bidx, n_batchs, gen_iterations,
                 D_mmd2.mean().data[0], G_mmd2.mean().data[0], mmd2_real.mean().data[0],
                 real_L2_loss.data[0], fake_L2_loss.data[0]))

        if gen_iterations % args.eval_freq == 0:
            # ========= Main block for evaluate MMD(X_p_enc, X_f_enc) on RNN codespace  =========#
            val_dict = valid_epoch(Data, Data.val_set, netD, args.batch_size, Y_val, L_val)
            tst_dict = valid_epoch(Data, Data.tst_set, netD, args.batch_size, Y_tst, L_tst)
            total_time = time.time() - start_time
            print('iter %4d tm %4.2fm val_mse %.1f val_mae %.1f val_auc %.6f'
                    % (epoch, total_time / 60.0, val_dict['mse'], val_dict['mae'], val_dict['auc']), end='')

            print (" tst_mse %.1f tst_mae %.1f tst_auc %.6f" % (tst_dict['mse'], tst_dict['mae'], tst_dict['auc']), end='')

            assert(np.isnan(val_dict['auc']) != True)
            #if val_dict['auc'] > best_val_auc:
            #if val_dict['auc'] > best_val_auc and mmd2_real.mean().data[0] < best_mmd_real:
            if mmd2_real.mean().data[0] < best_mmd_real:
                best_mmd_real = mmd2_real.mean().data[0]
                best_val_mae = val_dict['mae']
                best_val_auc = val_dict['auc']
                best_tst_auc = tst_dict['auc']
                best_epoch = epoch
                save_pred_name = '%s/pred.pkl' % (args.save_path)
                with open(save_pred_name, 'wb') as f:
                    pickle.dump(tst_dict, f)
                torch.save(netG.state_dict(), '%s/netG.pkl' % (args.save_path))
                torch.save(netD.state_dict(), '%s/netD.pkl' % (args.save_path))
            print(" [best_val_auc %.6f best_tst_auc %.6f best_epoch %3d]" % (best_val_auc, best_tst_auc, best_epoch))

        # stopping condition
        #if best_mmd_real < 1e-4:
        if mmd2_real.mean().data[0] < 1e-5:
            exit(0)

n_batchs 5 batch_size 128
start training: lambda_ae 0.001 lambda_real 0.1 weight_clip 0.1
G network


<ipython-input-90-9fe2a6c7bc76>:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  noise = Variable(noise, volatile=True) # total freeze netG
<ipython-input-85-dd83be09ac40>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sigma_samples = F.softmax(U * gumbel_lmd).matmul(sigma_var)


IndexError: ignored